In [1]:
import os.path

import requests
import pandas as pd

parent_dir = os.path.dirname(os.getcwd())

In [2]:
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN", "f964dcbdc8413c5d56c0ff2596bc9f75dd978ef966f431c991461e7dea1d656340c362975e34bac743fea")

In [3]:
group_name = "overhear"

In [4]:
VK_API_URL = "https://api.vk.com/method/{method}"

In [5]:
group_info = requests.get(url=VK_API_URL.format(method="groups.getById"),
                          params={
                              "v": 5.131,
                              "access_token": ACCESS_TOKEN,
                              "group_ids": group_name,
                              "fields": "description",
                          })

In [6]:
group_info.json()

{'response': [{'id': 34215577,
   'description': 'Мы публикуем только то, что присылают наши читатели.\n\nПоделиться откровением совершенно анонимно можно только с помощью нашего сайта https://ideer.ru/add или мобильного приложения https://ideer.app\n\n«Подслушано» — социальный развлекательный проект, в котором люди делятся каждый день своими секретами, откровениями и жизненными ситуациями анонимно перед огромной аудиторией. Добро пожаловать!\n\nЕсли вы впечатлительны и высокоморальны, просим вас не читать контент, который можно увидеть на стене сообщества, лучше сразу закройте эту страницу и не возвращайтесь сюда никогда :) \n\nТут особенная атмосфера и эмоции от прочитанного очень разнятся.\n\nЗачастую нас стоит читать между строк и никогда во время еды. \n\nЗдесь вы можете увидеть лишь малую часть того, что происходит в нашем мобильном приложении: https://ideer.app\n\nМнение редакции не всегда совпадает с мнением авторов.',
   'name': 'Подслушано – Здесь говорят о тебе',
   'screen_

In [7]:
group_id = group_info.json()["response"][0]["id"]

In [8]:
group_id

34215577

In [9]:
all_posts = 10000
loaded = 100
offset = loaded

In [10]:
info = requests.get(url=VK_API_URL.format(method="wall.get"),
                    params={
                        "v": 5.131,
                        "access_token": ACCESS_TOKEN,
                        "owner_id": -group_id,
                        "count": 100,
                        "offset": offset,
                    })

In [19]:
info.json()["response"]["items"][2]

{'id': 1674548,
 'from_id': -34215577,
 'owner_id': -34215577,
 'date': 1654343400,
 'marked_as_ads': 0,
 'is_favorite': False,
 'post_type': 'post',
 'text': 'Один мой отсаженный знакомый на похоронах своей дальней тётки сказал: "Бог дал — Бог взял", — после чего его же мама отпиздила его какой-то чёрной тряпкой и его выставили с поминок.\n\n#Подслушано_чернуха@overhear',
 'post_source': {'type': 'api'},
 'comments': {'can_post': 0,
  'count': 13,
  'donut': {'placeholder': {'text': 'Чтобы видеть больше комментариев и\xa0оставлять свои, поддержите сообщество.'}}},
 'likes': {'can_like': 1, 'count': 728, 'user_likes': 0, 'can_publish': 1},
 'reposts': {'count': 229, 'user_reposted': 0},
 'views': {'count': 94577},
 'donut': {'is_donut': False},
 'short_text_rate': 0.8,
 'hash': '9a4cAq64W0uk4pnophSXNe2rCygK'}

In [ ]:
posts = []

In [ ]:
def post_to_dict(item: dict) -> dict:
    return {"id": item["id"], "text": item["text"], "likes": item["likes"]["count"], "reposts": item["reposts"]["count"], "views": item["views"]["count"]}

while len(posts) <\
        100000:
    offset = len(posts)
    info = requests.get(url=VK_API_URL.format(method="wall.get"),
                    params={
                        "v": 5.131,
                        "access_token": ACCESS_TOKEN,
                        "owner_id": -group_id,
                        "count": 100,
                        "offset": offset,
                    })
    if info.status_code != 200:
        break
    try:
        for item in info.json()["response"]["items"]:
            posts.append(post_to_dict(item))
    except KeyError:
        continue
    finally:
        print(len(posts))

In [ ]:
posts[10:15]

In [ ]:
posts_df = pd.DataFrame(data=posts)

In [ ]:
posts_df.head()

In [ ]:
posts_df.to_csv(os.path.join(parent_dir, "datasets/posts.csv"), index=False)